In [1]:
#%ls ..\Exe

import os, flopy

#Going up to the example directory
exampleDir = os.path.dirname(os.getcwd())

simName = 'mf6uzfexamplesim'
simPath = exampleDir + '\Simulation'
exeName = exampleDir + '\Exe\mf6.exe'

sim = flopy.mf6.MFSimulation(sim_name=simName,
                            version='mf6', exe_name=exeName,
                            sim_ws=simPath)

Directory structure already exists for simulation path C:\Users\USER\Documents\Infohataris\BasicExampleMODFLOWUZFFlopy\Simulation


In [2]:

tdis = flopy.mf6.ModflowTdis(sim, pname='tdis', time_units='DAYS', nper=2, 
                             perioddata=[(1.0, 1, 1.0), (10.0, 5, 1.0)])

In [3]:
model_name = 'example_model'
model = flopy.mf6.MFModel(sim, modelname=model_name,
                             model_nam_file='{}.nam'.format(model_name))

In [4]:

ims_package  = flopy.mf6.ModflowIms(sim, pname='ims', print_option='ALL',
                                   complexity='SIMPLE', outer_hclose=0.00001,
                                   outer_maximum=50, under_relaxation='NONE',
                                   inner_maximum=30, inner_hclose=0.00001,
                                   linear_acceleration='CG',
                                   preconditioner_levels=7,
                                   preconditioner_drop_tolerance=0.01,
                                   number_orthogonalizations=2)

In [5]:
sim.register_ims_package(ims_package, [model_name])

In [6]:

dis_packagedis_pac  = flopy.mf6.ModflowGwfdis(model, pname='dis', length_units='FEET', nlay=2,
                                      nrow=2, ncol=5, delr=500.0,
                                      delc=500.0,
                                      top=100.0, botm=[50.0, 20.0],
                                      fname='{}.dis'.format(model_name))

In [7]:
# set the nocheck property in the simulation namefile
sim.name_file.nocheck = True
# set the print_input option in the model namefile
model.name_file.print_input = True

In [8]:
# Turn Newton option on with under relaxation 
model.name_file.newtonoptions = ('UNDER_RELAXATION')
# Turn Newton option on without under relaxation
model.name_file.newtonoptions = ('')
# Turn off Newton option 
model.name_file.newtonoptions = (None)

In [9]:
# build a data template for k that stores the first layer as an internal array and the second
# layer as a constant with the default value of k for all layers set to 100.0 
layer_storage_types = [flopy.mf6.data.mfdata.DataStorageType.internal_array, 
                       flopy.mf6.data.mfdata.DataStorageType.internal_constant]
k_template = flopy.mf6.ModflowGwfnpf.k.empty(model, True, layer_storage_types, 100.0)
# change the value of the second layer to 50.0
k_template[0]['data'] = [65.0, 60.0, 55.0, 50.0, 45.0, 40.0, 35.0, 30.0, 25.0, 20.0]
k_template[0]['factor'] = 1.5
print(k_template)
# create npf package using the k template to define k
npf_package = flopy.mf6.ModflowGwfnpf(model, pname='npf', save_flows=True, icelltype=1, k=k_template)

[{'factor': 1.5, 'iprn': 1, 'data': [65.0, 60.0, 55.0, 50.0, 45.0, 40.0, 35.0, 30.0, 25.0, 20.0]}, 100.0]


In [10]:
from shutil import copyfile
strt={'filename': 'strt.txt', 'factor':1.0, 'data':[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 
      90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0]}
ic_package = flopy.mf6.ModflowGwfic(model, pname='ic', strt=100,
                                    fname='{}.ic'.format(model_name))

# create storage package
sto_package = flopy.mf6.ModflowGwfsto(model, pname='sto', save_flows=True, iconvert=0,
                                      ss=[0.000001, 0.000002], 
                                      sy=[0.15, 0.14, 0.13, 0.12, 0.11, 0.11, 0.12, 0.13, 0.14, 0.15,
                                         0.15, 0.14, 0.13, 0.12, 0.11, 0.11, 0.12, 0.13, 0.14, 0.15])

In [11]:

maxbound  = 2
# build a stress_period_data template with 2 wells over stress periods 1 and 2 with boundnames 
# and three aux variables
wel_periodrec = flopy.mf6.ModflowGwfwel.stress_period_data.empty(model, maxbound=maxbound, boundnames=True, 
                                                             aux_vars=['var1', 'var2', 'var3'],
                                                             stress_periods=[0,1])
# define the two wells for stress period one
wel_periodrec[0][0] = ((0,1,2), -50.0, -1, -2, -3, 'First Well')
wel_periodrec[0][1] = ((1,1,4), -25.0, 2, 3, 4, 'Second Well')
# define the two wells for stress period two
wel_periodrec[1][0] = ((0,1,2), -200.0, -1, -2, -3, 'First Well')
wel_periodrec[1][1] = ((1,1,4), -4000.0, 2, 3, 4, 'Second Well')
# build the well package
wel_package = flopy.mf6.ModflowGwfwel(model, pname='wel', print_input=True, print_flows=True,
                                      auxiliary=[('var1', 'var2', 'var3')], maxbound=maxbound,
                                      stress_period_data=wel_periodrec, boundnames=True, save_flows=True)

In [12]:

# printrecord data as a list of tuples.  since no stress# print 
# period is specified it will default to stress period 1
printrec_tuple_list = [('HEAD', 'ALL'), ('BUDGET', 'ALL')]
# saverecord data as a dictionary of lists of tuples for 
# stress periods 1 and 2.  
saverec_dict = {0:[('HEAD', 'ALL'), ('BUDGET', 'ALL')],1:[('HEAD', 'ALL'), ('BUDGET', 'ALL')]}
# create oc package
oc_package = flopy.mf6.ModflowGwfoc(model, pname='oc', 
                                    budget_filerecord=[('{}.cbc'.format(model_name),)],
                                    head_filerecord=[('{}.hds'.format(model_name),)],
                                    saverecord=saverec_dict,
                                    printrecord=printrec_tuple_list)
# add stress period two to the print record
oc_package.printrecord.add_transient_key(1)
# set the data for stress period two in the print record
oc_package.printrecord.set_data([('HEAD', 'ALL'), ('BUDGET', 'ALL')], 1)

In [13]:
stress_period_data = {0: [[(0, 0, 0), 70.]],
                      1: [[(0, 0, 0), 60.]]}
chd = flopy.mf6.ModflowGwfchd(model, maxbound=1, stress_period_data=stress_period_data)

In [14]:
# write simulation to new location
sim.write_simulation()

# run simulation
sim.run_simulation()

INFORMATION: 0 external files copied
FloPy is using the following executable to run the model: C:\Users\USER\Documents\Infohataris\BasicExampleMODFLOWUZFFlopy\Exe\mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                       VERSION mf6.0.2 February 23, 2018

   MODFLOW 6 compiled Feb 21 2018 10:49:51 with IFORT compiler (ver. 18.0.1)

This software has been approved for release by the U.S. Geological 
Survey (USGS). Although the software has been subjected to rigorous 
review, the USGS reserves the right to update the software as needed 
pursuant to further analysis and review. No warranty, expressed or 
implied, is made by the USGS or the U.S. Government as to the 
functionality of the software and related material nor shall the 
fact of release constitute any such warranty. Furthermore, the 
software is released on condition that neither the USGS nor the U.S. 
Government shall be held liable for any damage

(True, [])